# Import Libraires

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot
import seaborn as sns
import scipy

# Import Data

In [2]:
path = r'/Users/docopeland/04 Instacart Basket Analysis'
final = pd.read_pickle(os.path.join(path,'02 Data','prepared data','orders_products_all.pkl'))
departments = pd.read_csv(os.path.join(path,'02 Data','prepared data','departments_clean.csv'))

# Deriving a region column

In [3]:
#creating a region column based on the state

In [4]:
#northeast region
northeast = ['Maine','New Hampshire','Vermont','Massachusetts','Rhode Island','Connecticut','New York',
             'Pennsylvania','New Jersey']

In [5]:
#midwest region
midwest = ['Wisconsin','Michigan','Illinois','Indiana','Ohio','North Dakota','South Dakota','Nebraska',
           'Kansas','Minnesota','Iowa','Missouri']

In [6]:
#south region
south = ['Delaware','Maryland','District of Columbia','Virginia','West Virginia','North Carolina',
         'South Carolina','Georgia','Florida','Kentucky','Tennessee','Mississippi','Alabama',
         'Oklahoma','Texas','Arkansas','Louisiana']

In [7]:
#west region
west = ['Idaho','Montana','Wyoming','Nevada','Utah','Colorado','Arizona','New Mexico','Alaska',
        'Washington','Oregon','California','Hawaii']

In [8]:
#create a list of regions
region = []
for val in final['state']:
    if val in northeast:
        region.append('Northeast')
    if val in midwest:
        region.append('Midwest')
    if val in south:
        region.append('South')
    if val in west:
        region.append('West')

In [9]:
final['region'] = region

# Deriving an exclusion flag

In [10]:
#create an exclusion flag based on the number of max orders
#if max_order is less than 5, exclusion is False
#if max_order is greater than or equal to 5, exclusion is True

In [11]:
#false exclusion
final.loc[final['max_order'] < 5, 'exclusion_flag'] = False

In [12]:
#true exclusion
final.loc[final['max_order'] >= 5, 'exclusion_flag'] = True

In [13]:
#check values
final['exclusion_flag'].value_counts()

True     30964564
False     1440295
Name: exclusion_flag, dtype: int64

In [14]:
included = final[final['exclusion_flag'] == True]

In [15]:
included['exclusion_flag'].value_counts()

True    30964564
Name: exclusion_flag, dtype: int64

In [16]:
included.shape

(30964564, 33)

# Creating profiles

In [17]:
#flags to create:
#age flag - young adult (18 to 25), adult (26 to 65), older adult (65+)
#parent flag - n_dependants > 0
#income flag - low income (0 to 67,292), middle income (67,292 to 128,102), high income (128,102+)
#order time of day flag - morning (0-11), afternoon (12-17), night (18-24)
#order time of week flag - weekened (0-1), weekday (2-6)

In [18]:
#age_flag for loop
adulting = []
for num in included['age']:
    if num <=25:
        adulting.append('young adult')
    elif num > 25 and num <65:
        adulting.append('mature adult')
    else:
        adulting.append('older adult')

In [19]:
#adding age flag to df
included['age_flag'] = adulting

/var/folders/zw/88f_mbgs7zz4slnzcgksqp2c0000gn/T/ipykernel_91080/1193966588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  included['age_flag'] = adulting


In [20]:
included['age_flag'].value_counts()

mature adult    18904502
older adult      8195544
young adult      3864518
Name: age_flag, dtype: int64

In [21]:
#parent_flag, parent
included.loc[included['n_dependants'] > 0, 'parent_flag'] = 'parent'

/Users/docopeland/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/docopeland/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [22]:
#parent_flag, not parent
included.loc[included['n_dependants'] == 0, 'parent_flag'] = 'not parent'

In [23]:
included['parent_flag'].value_counts()

parent        23224883
not parent     7739681
Name: parent_flag, dtype: int64

In [24]:
#income_flag, for loop
inco = []
for val in included['income']:
    if val <= 67292:
        inco.append('Low-income')
    elif val > 67292 and val <=128102:
        inco.append('Mid-income')
    else:
        inco.append('High-income')

In [25]:
#income_flag, add to df
included['income_flag'] = inco

/var/folders/zw/88f_mbgs7zz4slnzcgksqp2c0000gn/T/ipykernel_91080/2327613873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  included['income_flag'] = inco


In [26]:
included['income_flag'].value_counts()

Mid-income     15482125
Low-income      7741348
High-income     7741091
Name: income_flag, dtype: int64

In [27]:
#time_of_day, for loop
tod = []
for hour in included['order_hour_of_day']:
    if hour <= 11:
        tod.append('Morning')
    elif hour > 12 and hour <= 17:
        tod.append('Afternoon')
    else:
        tod.append('Night')

In [28]:
#time_of_day, add to df
included['time_of_day'] = tod

/var/folders/zw/88f_mbgs7zz4slnzcgksqp2c0000gn/T/ipykernel_91080/1005776706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  included['time_of_day'] = tod


In [29]:
included['time_of_day'].value_counts()

Afternoon    12039171
Morning      10998902
Night         7926491
Name: time_of_day, dtype: int64

In [30]:
#time_of_week, weekend
included.loc[included['order_day_of_week'] <= 1, 'time_of_week'] = 'Weekend'

In [31]:
#time_of_week, weekday
included.loc[included['order_day_of_week'] > 1 , 'time_of_week'] = "Weekday"

In [32]:
included['time_of_week'].value_counts()

Weekday    19644268
Weekend    11320296
Name: time_of_week, dtype: int64

In [33]:
#profiles to create
#single adult - !parent + (fam_status = single or divorced/widowed)
#single parent - parent + (fam_status = single or divorced/widowed)
#family - parent + (fam_status = married)
#couple - !parent + (fam_status = married)
#supported parent - parent + (fam_status = living with parents and siblings)
#supported adult - !parent + (fam_status = living with parents and siblings)

In [34]:
included['fam_status'].value_counts()

married                             21743711
single                               5094410
divorced/widowed                     2645271
living with parents and siblings     1481172
Name: fam_status, dtype: int64

In [35]:
included['parent_flag'].value_counts()

parent        23224883
not parent     7739681
Name: parent_flag, dtype: int64

In [36]:
#profile, for loop
pro = []
for par, fam in zip(included['parent_flag'],included['fam_status']):
    if (par == 'not parent') and (fam == 'single' or fam == 'divorced/widowed'):
        pro.append('single adult')
    elif (par == 'parent') and (fam == 'single' or fam == 'divorced/widowed'):
        pro.append('single parent')
    elif (par == 'parent') and (fam == 'married'):
        pro.append('family')
    elif (par == 'not parent') and (fam == 'married'):
        pro.append('couple')
    elif (par == 'parent') and (fam == 'living with parents and siblings'):
        pro.append('supported parent')
    else:
        pro.append('supported adult')

In [37]:
#adding pro to df
included['profile'] = pro

/var/folders/zw/88f_mbgs7zz4slnzcgksqp2c0000gn/T/ipykernel_91080/3177052029.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  included['profile'] = pro


In [38]:
#checking profile value counts
included['profile'].value_counts()

family              21743711
single adult         7739681
supported parent     1481172
Name: profile, dtype: int64

# Adding Department Data

In [39]:
departments.head()

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [40]:
#dropping first column
departments = departments.drop(columns = ['Unnamed: 0'])

In [41]:
#set the df to be a dictionary
depart_dict = departments.to_dict()

In [42]:
#since it's a nested dictionary, saving the nested part into another dict
d_dict = depart_dict.get('department')

In [43]:
#viewing the department_ids in the df
included['department_id'].value_counts().sort_index()

1     2121731
2       34411
3     1120828
4     9079273
5      144627
6      255991
7     2571901
8       93060
9      822136
10      33451
11     424306
12     674781
13    1782705
14     670850
15    1012074
16    5177182
17     699857
18     410392
19    2766406
20    1003834
21      64768
Name: department_id, dtype: int64

In [44]:
#checking to see if the d_dict.keys() match the department_ids, they don't, they're one lower
d_dict.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [45]:
#creating a list of the different departments, for loop
all_IDs = []
for ID in included['department_id']:
    a = d_dict.get(ID-1)
    all_IDs.append(a)

In [46]:
#adding the list to the df
included['department'] = all_IDs

/var/folders/zw/88f_mbgs7zz4slnzcgksqp2c0000gn/T/ipykernel_91080/3673058569.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  included['department'] = all_IDs


In [47]:
#check the department vavlues
included['department'].value_counts()

produce            9079273
dairy eggs         5177182
snacks             2766406
beverages          2571901
frozen             2121731
pantry             1782705
bakery             1120828
canned goods       1012074
deli               1003834
dry goods pasta     822136
household           699857
meat seafood        674781
breakfast           670850
personal care       424306
babies              410392
international       255991
alcohol             144627
pets                 93060
missing              64768
other                34411
bulk                 33451
Name: department, dtype: int64

In [48]:
#checking that the department_id value counts match up, they do
included['department_id'].value_counts()

4     9079273
16    5177182
19    2766406
7     2571901
1     2121731
13    1782705
3     1120828
15    1012074
20    1003834
9      822136
17     699857
12     674781
14     670850
11     424306
18     410392
6      255991
5      144627
8       93060
21      64768
2       34411
10      33451
Name: department_id, dtype: int64

# Exporting data

In [49]:
included.to_pickle(os.path.join(path,'02 Data','prepared data','final.pkl'))